In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
import requests
import random

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries notebook and jupyter are not installed in interpreter Python 3.9.0 64-bit.

In [ ]:
model_name = "774M"

In [ ]:
gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, model_name=model_name)

In [ ]:
def generateWithPrefix(sess, prefix, length=100, include_prefix=True):
  return gpt2.generate(sess,
              model_name=model_name,
              prefix=prefix,
              length=length,
              temperature=0.7,
              top_p=0.9,
              include_prefix=include_prefix,
              return_as_list=True
              )[0]

def getPartOfSpeech(word):
  result = requests.get("https://api.dictionaryapi.dev/api/v2/entries/en/" + word)

  if result.status_code != 200:
    return None
  
  json = result.json()
  return json[0]["meanings"][0]["partOfSpeech"]

# getPartOfSpeech("table")
def replacePartOfSpeech(sentence, word, partOfSpeech):
  sentence += " "
  output = ""
  currentWord = ""
  for char in sentence:
    if char >= 'a' and char <= 'z':
      currentWord += char
    else:
      if (len(word) > 0):
        # end of word
        wordPartOfSpeech = getPartOfSpeech(currentWord)
        if (wordPartOfSpeech == partOfSpeech and random.uniform(0, 1) > 0.5):
          output += word
          return output
        else:
          output += currentWord
        currentWord = ""
      output += char

  return None

def insertWordAndTruncate(sentence, word):
  partOfSpeech = getPartOfSpeech(word)
  return replacePartOfSpeech(sentence, word, partOfSpeech)

def generateWithKeywords(sess, prefix, keywords):
  output = prefix
  for word in keywords:
    attempts = 0
    addition = None
    while (attempts < 5 and addition == None):
      # output += " "
      print("Prefix: " + output)
      print("Word: " + word)
      generated = generateWithPrefix(sess, output, 20 + attempts * 10)
      print("Generated: " + generated)
      addition = insertWordAndTruncate(generated[len(output):], word)
      print("Truncated: " + str(addition))
      
      attempts += 1
    if (addition != None):
      output += addition
  
  output = generateWithPrefix(sess, output, 20)
  return output

In [ ]:
story = generateWithKeywords(sess, "Once upon a time, ", ["shower", "mountain", "gear"])
print(str(story))